In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataset-of-00s.csv to dataset-of-00s (1).csv
Saving dataset-of-10s.csv to dataset-of-10s (1).csv
Saving dataset-of-60s.csv to dataset-of-60s (1).csv
Saving dataset-of-70s.csv to dataset-of-70s (1).csv
Saving dataset-of-80s.csv to dataset-of-80s (1).csv
Saving dataset-of-90s.csv to dataset-of-90s (1).csv


In [2]:
import os
import glob
import pandas as pd
import numpy as np 

np.random.seed(20)

In [3]:
# Use glob to match the pattern ‘csv’
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['dataset-of-00s.csv',
 'dataset-of-60s (1).csv',
 'dataset-of-60s.csv',
 'dataset-of-70s.csv',
 'dataset-of-80s (1).csv',
 'dataset-of-10s (1).csv',
 'dataset-of-90s (1).csv',
 'dataset-of-80s.csv',
 'dataset-of-90s.csv',
 'dataset-of-00s (1).csv',
 'dataset-of-70s (1).csv',
 'dataset-of-10s.csv']

In [4]:
# Combine all files in the list and export as CSV
data = pd.concat([pd.read_csv(f) for f in all_filenames ])
data.reset_index(drop=True, inplace=True)
data

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target
0,Lucky Man,Montgomery Gentry,spotify:track:4GiXBCUF7H6YfNQsnBRIzl,0.578,0.471,4,-7.270,1,0.0289,0.368000,0.000000,0.159,0.5320,133.061,196707,4,30.88059,13,1
1,On The Hotline,Pretty Ricky,spotify:track:1zyqZONW985Cs4osz9wlsu,0.704,0.854,10,-5.477,0,0.1830,0.018500,0.000000,0.148,0.6880,92.988,242587,4,41.51106,10,1
2,Clouds Of Dementia,Candlemass,spotify:track:6cHZf7RbxXCKwEkgAZT4mY,0.162,0.836,9,-3.009,1,0.0473,0.000111,0.004570,0.174,0.3000,86.964,338893,4,65.32887,13,0
3,"Heavy Metal, Raise Hell!",Zwartketterij,spotify:track:2IjBPp2vMeX7LggzRN3iSX,0.188,0.994,4,-3.745,1,0.1660,0.000007,0.078400,0.192,0.3330,148.440,255667,4,58.59528,9,0
4,I Got A Feelin',Billy Currington,spotify:track:1tF370eYXUcWwkIvaq3IGz,0.630,0.764,2,-4.353,1,0.0275,0.363000,0.000000,0.125,0.6310,112.098,193760,4,22.62384,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82207,Lotus Flowers,Yolta,spotify:track:4t1TljQWJ6ZuoSY67zVvBI,0.172,0.358,9,-14.430,1,0.0342,0.886000,0.966000,0.314,0.0361,72.272,150857,4,24.30824,7,0
82208,Calling My Spirit,Kodak Black,spotify:track:2MShy1GSSgbmGUxADNIao5,0.910,0.366,1,-9.954,1,0.0941,0.099600,0.000000,0.261,0.7400,119.985,152000,4,32.53856,8,1
82209,Teenage Dream,Katy Perry,spotify:track:55qBw1900pZKfXJ6Q9A2Lc,0.719,0.804,10,-4.581,1,0.0355,0.013200,0.000003,0.139,0.6050,119.999,227760,4,20.73371,7,1
82210,Stormy Weather,Oscar Peterson,spotify:track:4o9npmYHrOF1rUxxTVH8h4,0.600,0.177,7,-16.070,1,0.0561,0.989000,0.868000,0.149,0.5600,120.030,213387,4,21.65301,14,0


In [5]:
data.columns

Index(['track', 'artist', 'uri', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'chorus_hit',
       'sections', 'target'],
      dtype='object')

In [6]:
y = data.target
x = data[['danceability', 'energy', 'key', 'loudness','mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness','valence', 'tempo', 'duration_ms', 'time_signature', 'chorus_hit','sections']]

In [7]:
print(x.shape, y.shape)

(82212, 15) (82212,)


In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=12)

### With RANDOM FOREST

In [9]:
from sklearn.preprocessing import StandardScaler

x_scaler = StandardScaler().fit(x_train)

x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [10]:
# Create a random forest classifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, max_depth=25) 
model = model.fit(x_train_scaled, y_train)

In [11]:
print(f"Training Data Score: {model.score(x_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(x_test_scaled, y_test)}")

Training Data Score: 0.9964482070743931
Testing Data Score: 0.9401060672407922


In [12]:
# Validation of feature importance
importances = model.feature_importances_
importances

array([0.10244483, 0.08098991, 0.02807624, 0.07954222, 0.01050716,
       0.0735539 , 0.11091336, 0.1721812 , 0.04944604, 0.0703247 ,
       0.0526965 , 0.07634634, 0.00681551, 0.04885519, 0.03730689])

In [13]:
# We can sort the features by their importance

sorted(zip(model.feature_importances_, x.columns), reverse=True)

[(0.17218120205900364, 'instrumentalness'),
 (0.11091336029836071, 'acousticness'),
 (0.1024448306178178, 'danceability'),
 (0.08098990914937784, 'energy'),
 (0.0795422221632958, 'loudness'),
 (0.07634634470391682, 'duration_ms'),
 (0.07355390414671903, 'speechiness'),
 (0.0703246953334037, 'valence'),
 (0.05269649915486119, 'tempo'),
 (0.0494460402718466, 'liveness'),
 (0.04885519419863075, 'chorus_hit'),
 (0.037306890328436516, 'sections'),
 (0.028076239604106643, 'key'),
 (0.010507160717155354, 'mode'),
 (0.006815507253067717, 'time_signature')]

In [14]:
# Save the model
from google.colab import drive
drive.mount('/content/gdrive')
path = F"/content/gdrive/My Drive/rf.h5" 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
import pickle
with open(path, 'wb') as f:
    pickle.dump(model, f)